In [ ]:
#https://tykimos.github.io/2017/03/08/CNN_Getting_Started/

In [ ]:
#!\rm -rf /content/tr_images.zip
#!\rm -rf /content/img_gen
#!\rm -rf /content/image
!\rm -rf /content/data
#!\rm -rf /content/train/rectangle
!mkdir /content/data
!mkdir /content/data/train
!mkdir /content/data/train/circle
!mkdir /content/data/train/rectangle
!mkdir /content/data/train/triangle
!mkdir /content/data/test
!mkdir /content/data/test/circle
!mkdir /content/data/test/rectangle
!mkdir /content/data/test/triangle

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [ ]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# 학습용 데이터 부풀리기 - 샘플 이미지를 돌리고 비틀어서
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=10,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

In [ ]:
# 제네레이터로 생성된 배치로 학습시킬 경우 fit() 대신 fit_generator() 함수를 사용
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=50,
        validation_data=test_generator
        ,validation_steps=5
    )

# 제네레이터로 생성된 배치로 검증할 경우 evaluate() 대신 evaluate_generator() 함수를 사용
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 역시 predict() 대신 predict_generator() 사용
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

print(test_generator.filenames)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
15/15 [==============================] - 1s 19ms/step - loss: 1.5731 - accuracy: 0.3070 - val_loss: 1.0869 - val_accuracy: 0.3333
Epoch 2/50
15/15 [==============================] - 0s 8ms/step - loss: 1.1001 - accuracy: 0.3440 - val_loss: 1.0794 - val_accuracy: 0.3333
Epoch 3/50
15/15 [==============================] - 0s 9ms/step - loss: 1.0505 - accuracy: 0.7091 - val_loss: 1.1002 - val_accuracy: 0.3333
Epoch 4/50
15/15 [==============================] - 0s 9ms/step - loss: 0.9458 - accuracy: 0.6493 - val_loss: 1.1401 - val_accuracy: 0.4000
Epoch 5/50
15/15 [==============================] - 0s 10ms/step - loss: 0.9093 - accuracy: 0.5454 - val_loss: 1.2029 - val_accuracy: 0.3333
Epoch 6/50
15/15 [==============================] - 0s 10ms/step - loss: 0.9318 - accuracy: 0.6284 - val_loss: 1.3382 - val_accuracy: 0.4000
Epoch 7/50
15/15 [==============================] - 0s 9ms/step - loss: 0.8243 - accuracy: 0.5452 - val_loss: 1.5428 - val_accuracy: 0.3333
Epoch 8/50
15/15 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
